# DATA 512 Project
## RIley Waters

In this notebook I will use City of Calgary Census data to answer the following questions:

*How do construction trends differ by community?

*What construction trends are occurring on the city-level?

*What vacancy trends are there across communities?

*How do demographics differ by community?

*How has community density and population changed over time?

## Data Aquisition
Three data sources are used. The first is the Census by Community 2019 dataset found here: https://data.calgary.ca/Demographics/Census-by-Community-2019/rkfr-buzb.

In [26]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)

# Read in the 2019 dataset
census2019_df = pd.read_csv('./data/Census_by_Community_2019.csv')
census2019_df.head()

,CLASS,CLASS_CODE,COMM_CODE,NAME,SECTOR,SRG,COMM_STRUCTURE,CNSS_YR,FOIP_IND,RES_CNT,...,OTHER_5_14,OTHER_15_19,OTHER_20_24,OTHER_25_34,OTHER_35_44,OTHER_45_54,OTHER_55_64,OTHER_65_74,OTHER_75,multipolygon
0,Residential,1,LEG,LEGACY,SOUTH,DEVELOPING,BUILDING OUT,2019,NaN,6420,...,0,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-114.021996041091 50.863078904...
1,Residential,1,HPK,HIGHLAND PARK,CENTRE,BUILT-OUT,1950s,2019,NaN,3838,...,0,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-114.0691626854784 51.09565033...
2,Residential,1,CNS,CORNERSTONE,NORTHEAST,DEVELOPING,2000s,2019,NaN,2648,...,0,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-113.91839732026011 51.1760690...
3,Residential,1,MON,MONTGOMERY,NORTHWEST,BUILT-OUT,1950s,2019,NaN,4515,...,0,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-114.16457918083577 51.0814533...
4,Residential,1,TEM,TEMPLE,NORTHEAST,BUILT-OUT,1960s/1970s,2019,NaN,10977,...,0,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-113.93512706147847 51.0960756...


The second is the Civic Census Results 1958-2019 Dataset found here: https://data.calgary.ca/Demographics/Civic-Census-Results-1958-2019/rmai-qvzh

In [2]:
# Read in the historical dataset
historical_df = pd.read_csv('./data/Civic_Census_Results_1958-2019.csv')
historical_df.head()

,Year,Ward,Dwellings,Residents,Vehicles,Age 65 +,Voters,CENSUS_YEAR
0,04/01/1958 12:00:00 AM,NaN,64288.0,206831,NaN,NaN,NaN,1958
1,04/01/1959 12:00:00 AM,NaN,NaN,218418,NaN,NaN,121813.0,1959
2,04/01/1960 12:00:00 AM,NaN,72987.0,235428,NaN,NaN,NaN,1960
3,04/01/1961 12:00:00 AM,1.0,NaN,38462,11455.0,NaN,21784.0,1961
4,04/01/1961 12:00:00 AM,2.0,NaN,35935,10201.0,NaN,19782.0,1961


The third is the Community Boudaries Layer. This is found here: https://data.calgary.ca/Base-Maps/Community-Boundaries/ab7m-fwn6.

In [31]:
map_df = pd.read_csv('./data/Community_Boundaries_layer.csv')

map_df.head()

,the_geom,CLASS,CLASS_CODE,COMM_CODE,NAME,SECTOR,SRG,COMM_STRUCTURE
0,"POLYGON ((-114.10061493005 51.047994301993, -1...",Residential,1,SNA,SUNALTA,CENTRE,BUILT-OUT,INNER CITY
1,"POLYGON ((-114.187805990867 51.061506344418, -...",Residential,1,WSP,WEST SPRINGS,WEST,DEVELOPING,BUILDING OUT
2,"POLYGON ((-113.869446537594 50.979572332142, -...",Residual Sub Area,4,12A,12A,SOUTHEAST,NaN,OTHER
3,"POLYGON ((-114.081517547811 51.008716296692, -...",Residential,1,WND,WINDSOR PARK,CENTRE,BUILT-OUT,1950s
4,"POLYGON ((-114.141027621875 51.074356354528, -...",Residential,1,UOC,UNIVERSITY OF CALGARY,NORTHWEST,BUILT-OUT,OTHER


## Combining the map and 2019 census
We merge the map dataset and the census dataset to allow for mapping with folium

In [34]:
map_df = map_df[['NAME', 'the_geom']]
census2019 = pd.merge(census2019_df, map_df, on='NAME', how='inner')
census2019.head()

,CLASS,CLASS_CODE,COMM_CODE,NAME,SECTOR,SRG,COMM_STRUCTURE,CNSS_YR,FOIP_IND,RES_CNT,...,OTHER_15_19,OTHER_20_24,OTHER_25_34,OTHER_35_44,OTHER_45_54,OTHER_55_64,OTHER_65_74,OTHER_75,multipolygon,the_geom
0,Residential,1,LEG,LEGACY,SOUTH,DEVELOPING,BUILDING OUT,2019,NaN,6420,...,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-114.021996041091 50.863078904...,"POLYGON ((-114.021996041091 50.86307890478, -1..."
1,Residential,1,HPK,HIGHLAND PARK,CENTRE,BUILT-OUT,1950s,2019,NaN,3838,...,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-114.0691626854784 51.09565033...,"POLYGON ((-114.069162685478 51.095650336941, -..."
2,Residential,1,CNS,CORNERSTONE,NORTHEAST,DEVELOPING,2000s,2019,NaN,2648,...,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-113.91839732026011 51.1760690...,"POLYGON ((-113.91839732026 51.176069026251, -1..."
3,Residential,1,MON,MONTGOMERY,NORTHWEST,BUILT-OUT,1950s,2019,NaN,4515,...,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-114.16457918083577 51.0814533...,"POLYGON ((-114.164579180836 51.081453378966, -..."
4,Residential,1,TEM,TEMPLE,NORTHEAST,BUILT-OUT,1960s/1970s,2019,NaN,10977,...,0,0,0,0,0,0,0,0,MULTIPOLYGON (((-113.93512706147847 51.0960756...,"POLYGON ((-113.935127061478 51.096075654696, -..."


## Looking at construction by community in 2019
To analyze which communities are building the most of each building type, we will look at the '{type}_UC' fields that indicate the number of buildings under construction for each building type.

Some communities are larger, so we will find the ratio of dwellings under construction to the total number of dwellings in each. We will also construct a ratio of the number of each type of dwelling presently in each community to the total of dwellings. This may uncover some communities that are building types they don't previously have many of.

### Apartments

In [45]:
# Filter to the columns of interest
APT_df = census2019_df[['NAME', 'CLASS', 'DWELL_CNT', 'APT_OCCPD', 'APT_VACANT', 'APT_UC', 'multipolygon']]

# Get the type ratio
APT_df['apartment_ratio'] = APT_df.eval('(APT_OCCPD+APT_VACANT)/DWELL_CNT')

# Get the construction ratio
APT_df['apt_uc_ratio'] = APT_df.eval('APT_UC/DWELL_CNT')

# Look at the top under-construction ratio
APT_df.sort_values('apt_uc_ratio', ascending=False).reset_index(drop=True).head(10)

,NAME,CLASS,DWELL_CNT,APT_OCCPD,APT_VACANT,APT_UC,multipolygon,apartment_ratio,apt_uc_ratio
0,SETON,Residential,973,247,105,256,MULTIPOLYGON (((-113.92944983055934 50.8707822...,0.361768,0.263104
1,CORNERSTONE,Residential,1285,0,0,219,MULTIPOLYGON (((-113.91839732026011 51.1760690...,0.000000,0.170428
2,MANCHESTER,Residential,659,487,19,109,MULTIPOLYGON (((-114.06172345801747 51.0160014...,0.767830,0.165402
3,DOWNTOWN WEST END,Residential,2061,1611,133,250,MULTIPOLYGON (((-114.0831727947718 51.05014640...,0.846191,0.121300
4,DOWNTOWN EAST VILLAGE,Residential,2699,1998,284,223,MULTIPOLYGON (((-114.05487576077839 51.0504023...,0.845498,0.082623
5,EAU CLAIRE,Residential,1355,1126,75,85,MULTIPOLYGON (((-114.069641108019 51.056777251...,0.886347,0.062731
6,MAHOGANY,Residential,4507,633,16,260,MULTIPOLYGON (((-113.92905647803371 50.9068525...,0.143998,0.057688
7,WEST SPRINGS,Residential,3856,97,81,216,MULTIPOLYGON (((-114.18780599086739 51.0615063...,0.046162,0.056017
8,SHAWNEE SLOPES,Residential,848,133,13,41,MULTIPOLYGON (((-114.09376274600558 50.9307576...,0.172170,0.048349
9,SOUTH CALGARY,Residential,2568,882,55,122,MULTIPOLYGON (((-114.09469234923867 51.0240025...,0.364875,0.047508


In [46]:
import folium

m = folium.Map(
    location = [51.044281, -114.062544],
    tiles = 'cartodbpositron',
    zoom_start=12
)
folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=APT_df,
    columns=['multipolygon', 'apt_uc_ratio'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)
m

NameError: name 'state_geo' is not defined